In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [2]:
df = pd.read_csv("./train.csv")
df.head()

X = df.drop(columns='label').values
y = df['label'].values

In [3]:
df.isnull().sum().sum()

0

In [7]:
model = Sequential([
  Input(shape=(784,)),
  Dense(units=50, activation="relu", name="Layer1"),
  Dense(units=30, activation="relu", name="Layer2"),
  Dense(units=15, activation="relu", name="Layer3"),
  Dense(units=10, activation="linear", name="Layer4"), # Will be using model.compile(loss=SparseCategoricalCrossentropy(from_logits=True))
])

In [8]:
print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Layer1 (Dense)                  │ (None, 50)             │        39,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Layer2 (Dense)                  │ (None, 30)             │         1,530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Layer3 (Dense)                  │ (None, 15)             │           465 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Layer4 (Dense)                  │ (None, 10)             │           160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,405 (161.74 KB)

 Trainable params: 41,405 (161.74 KB)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
weights = model.get_weights()

for weight in weights:
  print(len(weight), weight.shape)

784 (784, 50)
50 (50,)
50 (50, 30)
30 (30,)
30 (30, 15)
15 (15,)
15 (15, 10)
10 (10,)


In [31]:
model.compile(
    optimizer="nadam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
    jit_compile=True,
)

In [32]:
model.fit(X, y, epochs=50)

Epoch 1/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9936 - loss: 0.0294
Epoch 2/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9949 - loss: 0.0216
Epoch 3/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9930 - loss: 0.0302
Epoch 4/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9963 - loss: 0.0136
Epoch 5/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9929 - loss: 0.0344
Epoch 6/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9953 - loss: 0.0189
Epoch 7/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9933 - loss: 0.0310
Epoch 8/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9932 - loss: 0.0292
Epoch 9/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9960 - loss: 0.0166
Epoch 10/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9935 - loss: 0.0327
Epoch 11/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9954 - loss: 0.0183
Epoch 12/50
1313/1313 ━━━━━━━━

In [12]:
logits = model.predict(X)

probabilities = tf.nn.softmax(logits).numpy()
predicted_class = probabilities.argmax(axis = -1)

validation: np.ndarray = (predicted_class == y)
wrong = np.count_nonzero(validation == False)
print(f"Wrong: {wrong}, Correct: {df.shape[0] - wrong}")

1313/1313 ━━━━━━━━━━━━━━━━━━━━ 1s 662us/step
Wrong: 224, Correct: 41776


In [13]:
## Test data -> 
test = pd.read_csv("test.csv")
test.head()

testData = test.values

In [14]:
output = model.predict(testData)
predicted_class = probabilities.argmax(axis = -1)

predicted_class

875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 689us/step


array([1, 0, 1, ..., 7, 6, 9], dtype=int64)